In [122]:
import qmps
import xmps
import cirq
from xmps import iMPS
import numpy as np

In [123]:
# Initial State |ψ> = |0101010...>
# transfer matrix looks like ;
# A[σ, i, j] = [σᵃ, σᵇ] where σᵃ & σᵇ = |0X1| & |1X0| respectively 

A = np.random.randn(2,2,2)
A[0,:,:] = np.array([[0,1], [0,0]])
A[1,:,:] = np.array([[0,0], [1,0]])

from qmps.tools import tensor_to_unitary, environment_to_unitary
from qmps.represent import get_env_exact 
U = tensor_to_unitary(A)
U

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]])

In [124]:
V = 1/np.sqrt(2)*np.array([[1,1,0,0],[0,0,1,1],[0,0,1,-1],[1,-1,0,0]])
V

array([[ 0.70710678,  0.70710678,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.70710678,  0.70710678],
       [ 0.        ,  0.        ,  0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678,  0.        ,  0.        ]])

In [125]:
# Check that this analytical result hold in the simulator
from qmps.represent import Tensor

def simulate_state(gates, qubits, _print = False):
    qbs = cirq.LineQubit.range(qubits)
    circuit = cirq.Circuit.from_ops([gate(*qbs[-2:]) if i == 0 else gate(*qbs[-2-i:-i]) for i, gate in enumerate(gates)])
    
    if _print:
        print(circuit)
        
    sim = cirq.Simulator()
    res = sim.simulate(circuit)
    density_matrix = res.density_matrix_of(qbs[0:1])
    return density_matrix

state = Tensor(U, "U")
env = Tensor(V, "V")

gates = [env, state]
ρ_full = simulate_state(gates, 3, True)
print("-------------------------------------")
gate = [env]
ρ = simulate_state(gate, 2, True)
print("-------------------------------------")
print(ρ_full, "\n\n", ρ)

0: ───────U───
          │
1: ───V───U───
      │
2: ───V───────
-------------------------------------
0: ───V───
      │
1: ───V───
-------------------------------------
[[0.49999997+0.j 0.        +0.j]
 [0.        +0.j 0.49999997+0.j]] 

 [[0.49999997+0.j 0.        +0.j]
 [0.        +0.j 0.49999997+0.j]]


In [126]:
# Build the Hamiltonian for scar state evolution:
from scipy.linalg import expm

P = np.array([[1,0],[0,0]])
X = np.array([[0,1],[1,0]])
H = np.kron(np.kron(P,X),P)

dt = 0.1
W = Tensor(expm(-1j * H * dt), "W")



In [197]:
from qmps.represent import FullStateTensor, FullEnvironment, get_env_exact, Environment
from qmps.tools import unitary_to_tensor, environment_from_unitary, tensor_to_unitary
from xmps.spin import U4
from xmps.iMPS import Map, iMPS
from qmps.time_evolve_tools import merge, put_env_on_left_site, put_env_on_right_site
 
params = np.random.randn(15)
A = iMPS().random(2, 2).left_canonicalise()[0]
U = FullStateTensor(tensor_to_unitary(A))
W = Tensor(np.eye(8), "W")
    
# U_ = FullStateTensor(U4(params))
# U = FullStateTensor(tensor_to_unitary(A))
# b = unitary_to_tensor(cirq.unitary(U_))

_, r = Map(A,A).right_fixed_point()
_, l = Map(A,A).left_fixed_point()

L = Tensor(put_env_on_left_site(r.conj().T), "L")
R = Tensor(put_env_on_right_site(r), "R")

q = cirq.LineQubit.range(7)

circuit = cirq.Circuit.from_ops([
    cirq.H(q[4]),
    cirq.CNOT(q[4], q[5]),
    U(*q[3:5]),
    U(*q[2:4]),
    U(*q[1:3]),
    L(*q[0:2]),
    R(*q[5:7]),
    W(*q[2:5]),
    cirq.inverse(U)(*q[1:3]),
    cirq.inverse(U)(*q[2:4]),
    cirq.inverse(U)(*q[3:5]),
    cirq.CNOT(q[4], q[5]),
    cirq.H(q[4]),
    cirq.measure(*q)
])

print(circuit)
sim = cirq.Simulator()

result = sim.run(circuit, repetitions = 10000)


score = result.measurements

result_array = score[list(score.keys())[0]]
post_select = [res[1:-1] for res in result_array if res[0] == False & res[-1] == False]

all_zeros = [1 if sum(ps) == False else 0 for ps in post_select]
# this works if we set L = R = I, but never if we set R/L = r/l
print(np.average(all_zeros))

# n = np.trace(cirq.unitary(L).conj().T@cirq.unitary(R))
# print("n:", n)
# print(f"normalized: {np.average(all_zeros) / np.abs(n)}")

0: ───────────────────────L──────────────────────────M───
                          │                          │
1: ───────────────────U───L───U†─────────────────────M───
                      │       │                      │
2: ───────────────U───U───W───U†───U†────────────────M───
                  │       │        │                 │
3: ───────────U───U───────W────────U†───U†───────────M───
              │           │             │            │
4: ───H───@───U───────────W─────────────U†───@───H───M───
          │                                  │       │
5: ───────X───R──────────────────────────────X───────M───
              │                                      │
6: ───────────R──────────────────────────────────────M───
0.09516616314199396


In [201]:
ap(np.trace(cirq.unitary(L)@cirq.unitary(R))) 

0.9


In [154]:
ap(r, True)
print("----------------------")
ap(cirq.unitary(R), True)
print("----------------------")
ap(l, True)
print("----------------------")
ap(cirq.unitary(L), False) 

[[0.79 0.  ]
 [0.   0.62]]
----------------------
[[0.79 0.   0.62 0.  ]
 [0.   0.62 0.   0.79]
 [0.62 0.   0.79 0.  ]
 [0.   0.79 0.   0.62]]
----------------------
[[0.71 0.  ]
 [0.   0.71]]
----------------------
[[ 0.77+0.15j -0.  -0.j    0.  -0.j    0.61-0.12j]
 [ 0.  -0.j    0.58-0.22j  0.79+0.02j  0.  -0.j  ]
 [-0.  +0.j   -0.77+0.15j  0.61+0.12j -0.  -0.j  ]
 [-0.58-0.22j  0.  +0.j   -0.  -0.j    0.79-0.02j]]


In [100]:
def ap(A, _abs = True):
    if _abs:
        print(np.around(np.abs(A), 2))
    else:
        print(np.around(A, 2))
ap(cirq.unitary(U)@cirq.unitary(cirq.inverse(U)))

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [16]:
from qmps.represent import FullStateTensor, FullEnvironment, get_env_exact, Environment
from qmps.tools import unitary_to_tensor, environment_from_unitary, tensor_to_unitary
from xmps.spin import U4
from xmps.iMPS import Map, iMPS
from qmps.time_evolve_tools import merge, put_env_on_left_site, put_env_on_right_site

def scar_obj_fun(params, A, W):
    
    U_ = FullStateTensor(U4(params))
    U = FullStateTensor(tensor_to_unitary(A))
    b = unitary_to_tensor(cirq.unitary(U_))
    
    _, r = Map(A,A).right_fixed_point()
    _, l = Map(A,A).left_fixed_point()
    
    L = FullStateTensor(put_env_on_left_site(l))
    R = FullStateTensor(put_env_on_right_site(r))

    q = cirq.LineQubit.range(7)
    
    circuit = cirq.Circuit.from_ops([
        cirq.H(q[4]),
        cirq.CNOT(q[4], q[5]),
        U(*q[3:5]),
        U(*q[2:4]),
        U(*q[1:3]),
        L(*q[0:2]),
        R(*q[5:7]),
        W(*q[2:5]),
        cirq.inverse(U)(*q[1:3]),
        cirq.inverse(U)(*q[2:4]),
        cirq.inverse(U)(*q[3:5]),
        cirq.CNOT(q[4], q[5]),
        cirq.H(q[4]),
        cirq.measure(*q)
    ])
    
    print(circuit)
    sim = cirq.Simulator()

    result = sim.run(circuit, repetitions = 10000)
    
    
    return result.measurements

params = np.random.randn(15)
U = U4(params)
A = unitary_to_tensor(U)
W = Tensor(np.eye(8), "W")

score = scar_obj_fun(params, A, W)
result_array = score[list(score.keys())[0]]
post_select = [res[1:-1] for res in result_array if res[0] == 0 & res[-1] == 0]

all_zeros = [1 if sum(ps) == 0 else 0 for ps in post_select]
np.average(all_zeros)

0: ───────────────────────U──────────────────────────M───
                          │                          │
1: ───────────────────U───U───U†─────────────────────M───
                      │       │                      │
2: ───────────────U───U───W───U†───U†────────────────M───
                  │       │        │                 │
3: ───────────U───U───────W────────U†───U†───────────M───
              │           │             │            │
4: ───H───@───U───────────W─────────────U†───@───H───M───
          │                                  │       │
5: ───────X───U──────────────────────────────X───────M───
              │                                      │
6: ───────────U──────────────────────────────────────M───


0.20688960515713134

In [14]:
post_select

[array([ True,  True,  True,  True,  True]),
 array([ True, False, False, False,  True]),
 array([ True,  True,  True,  True,  True]),
 array([ True, False, False, False,  True]),
 array([ True,  True,  True, False,  True]),
 array([False, False, False, False,  True]),
 array([ True,  True, False,  True,  True]),
 array([False, False, False, False,  True]),
 array([ True, False, False,  True,  True]),
 array([False, False, False,  True,  True]),
 array([ True,  True,  True, False, False]),
 array([False, False, False,  True,  True]),
 array([False, False, False, False,  True]),
 array([False, False, False, False,  True]),
 array([False, False, False,  True,  True]),
 array([ True, False,  True, False,  True]),
 array([ True,  True, False,  True, False]),
 array([False, False, False,  True,  True]),
 array([ True, False, False, False,  True]),
 array([False, False, False, False,  True]),
 array([ True, False, False,  True, False]),
 array([ True,  True,  True, False, False]),
 array([ T

In [144]:
adj_probs = [score[i,i] for i in np.arange(0,score.shape[0]/2, 2, dtype = int)] 
adj_probs /= sum(adj_probs)
adj_probs
max(adj_probs)

(0.1302583+0j)

In [138]:
np.arange(0,score.shape[0]/2, 2)

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20., 22., 24.,
       26., 28., 30., 32., 34., 36., 38., 40., 42., 44., 46., 48., 50.,
       52., 54., 56., 58., 60., 62.])